# Simplify model definition with Sequential Model

In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable

In [3]:
# 1 create model
net = nn.Sequential(
    nn.Linear(28*28, 256),
    nn.ReLU(),
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Linear(128, 10) 
)
print(net)

Sequential(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=128, bias=True)
  (3): ReLU()
  (4): Linear(in_features=128, out_features=10, bias=True)
)


In [8]:
# 2 load data
import torchvision
import torch.utils.data as Data

train_data = torchvision.datasets.MNIST(
    root='./dataset/', 
    train=True,
    transform=torchvision.transforms.ToTensor())

test_data = torchvision.datasets.MNIST(
    root='./dataset/', 
    train=False)

train_data.data[0]

# batch train data
train_loader = Data.DataLoader(dataset=train_data, batch_size=32, shuffle=True)

# preprocess test data
# size: (n, 28, 28) -> (n, 28*28)
# value [0, 255] -> [0, 1]
with torch.no_grad():
    test_x = Variable(test_data.data.view(-1, 28*28)).type(torch.FloatTensor) / 255.0
test_y = test_data.targets

In [10]:
# 3 train and evaluate model
fun_loss = nn.CrossEntropyLoss() # cross entropy loss
optimizer = torch.optim.SGD(net.parameters(), lr=0.02) # SGD Optimizer

def evaluate(x, y):
    '''
    x: (n, 1, 28, 28)
    y: (n, 10)
    '''
    out = net(x)
    y_ = torch.max(out, 1)[1].detach() # max() return (value, index)
    accuracy = sum(y_==y) / y.size(0)
    return accuracy.item(), y_


# training and testing
for epoch in range(3):
    for step, (x, y) in enumerate(train_loader): 
        b_x = Variable(x.view(x.size(0), -1))   # batch x
        b_y = Variable(y)   # batch y

        output = net(b_x)               # ann output
        loss = fun_loss(output, b_y)    # cross entropy loss
        optimizer.zero_grad()           # clear gradients for this training step
        loss.backward()                 # backpropagation, compute gradients
        optimizer.step()                # apply gradients

        if step%50 == 0:
            accuracy, _ = evaluate(test_x, test_y)
            print(f'epoch: {epoch} | loss: {loss.detach().item()} | accuracy: {accuracy}')

epoch: 0 | loss: 0.41487565636634827 | accuracy: 0.9399999976158142
epoch: 0 | loss: 0.1505737155675888 | accuracy: 0.9434000253677368
epoch: 0 | loss: 0.2794940769672394 | accuracy: 0.9419000148773193
epoch: 0 | loss: 0.06305766850709915 | accuracy: 0.9380000233650208
epoch: 0 | loss: 0.09588237851858139 | accuracy: 0.9398999810218811
epoch: 0 | loss: 0.09343711286783218 | accuracy: 0.9444000124931335
epoch: 0 | loss: 0.18010397255420685 | accuracy: 0.9422000050544739
epoch: 0 | loss: 0.1423468291759491 | accuracy: 0.9447000026702881
epoch: 0 | loss: 0.03807302191853523 | accuracy: 0.9437999725341797
epoch: 0 | loss: 0.09074410796165466 | accuracy: 0.9462000131607056
epoch: 0 | loss: 0.39455974102020264 | accuracy: 0.9476000070571899
epoch: 0 | loss: 0.07555022835731506 | accuracy: 0.9470999836921692
epoch: 0 | loss: 0.14707492291927338 | accuracy: 0.9462000131607056
epoch: 0 | loss: 0.19503110647201538 | accuracy: 0.9463000297546387
epoch: 0 | loss: 0.09398728609085083 | accuracy: 0.